In [1]:
import numpy as np
import mininn
from pathlib import Path

np.set_printoptions(precision=4, suppress=True)
OUTPUT_DIR = Path("../output").resolve()
DATA_DIR = OUTPUT_DIR / "data"

In [2]:
def train(model, loss, train_loader, optimizer, epoch):
    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad();
        inputs = inputs.numpy(force=True)
        targets = targets.numpy(force=True)
        loss_output = loss.forward(model(inputs), targets)
        model.backward(loss.backward())
        optimizer.step()
        if (i % 10000 == 0):
            print(f"Epoch {epoch} loss {np.array(loss_output):.5f}")


In [3]:
from torchvision import datasets
from torch.utils.data import DataLoader

mnist_train = datasets.MNIST(
    DATA_DIR, 
    download=True, 
    transform=lambda x: np.array(x).reshape(28*28) / 255.0, 
)
mnist_test = datasets.MNIST(
    DATA_DIR,
    train=False,
    download=True,
    transform=lambda x: np.array(x).reshape(28*28) / 255.0,
)
train_loader = DataLoader(mnist_train, batch_size=8, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=8, shuffle=False)

model = mininn.Sequential([
    mininn.Linear(28 * 28, 200),
    mininn.ReLU(),
    mininn.Linear(200, 20),
])
loss = mininn.CrossEntropyLoss()
optimizer = mininn.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    train(model, loss, train_loader, optimizer, epoch)

Epoch 0 loss 2.96173
Epoch 1 loss 0.02652
Epoch 2 loss 0.01384
Epoch 3 loss 0.01200
Epoch 4 loss 0.00017
Epoch 5 loss 0.00009
Epoch 6 loss 0.00428
Epoch 7 loss 0.00086
Epoch 8 loss 0.00000
Epoch 9 loss 0.01009
Epoch 10 loss 0.00002
Epoch 11 loss 0.00002
Epoch 12 loss 0.00000
Epoch 13 loss 0.00080
Epoch 14 loss 0.00000
Epoch 15 loss 0.00030
Epoch 16 loss 0.00000
Epoch 17 loss 0.00036
Epoch 18 loss 0.00007
Epoch 19 loss 0.00000
Epoch 20 loss 0.00015
Epoch 21 loss 0.00001
Epoch 22 loss 0.00000
Epoch 23 loss 0.00000
Epoch 24 loss 0.00000
Epoch 25 loss 0.00000
Epoch 26 loss 0.15492
Epoch 27 loss 0.00000
Epoch 28 loss 0.00000
Epoch 29 loss 0.00000
Epoch 30 loss 0.00000
Epoch 31 loss 0.00000
Epoch 32 loss 0.00008
Epoch 33 loss 0.00000
Epoch 34 loss 0.00000
Epoch 35 loss 0.00000
Epoch 36 loss 0.00000
Epoch 37 loss 0.00000
Epoch 38 loss 0.00000
Epoch 39 loss 0.00000
Epoch 40 loss 0.00000
Epoch 41 loss 0.00000
Epoch 42 loss 0.00100
Epoch 43 loss 0.00002
Epoch 44 loss 0.00000
Epoch 45 loss 0.0000

In [4]:
def test(model, test_loader):
    num_correct = 0
    num_samples = 0
    for inputs, targets in test_loader:
        inputs = inputs.numpy(force=True)
        targets = targets.numpy(force=True)
        outputs = model(inputs)
        predictions = outputs.argmax(axis=1)
        num_correct += (predictions == targets).sum()
        num_samples += predictions.shape[0]
    print(num_correct / num_samples)

test(model, test_loader)

0.981


In [5]:
test(model, train_loader)

0.9995833333333334
